# Siamese Network + SVM with Raw Image

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os
def load_list(data_dir, prefix = None, dataset = 1):
    train_list = os.path.join(data_dir, 'train') + str(dataset) + '.idx'
    test_list = os.path.join(data_dir, 'test') + str(dataset) + '.idx'
    with open(train_list) as f:
        train_list = f.read().splitlines()
    with open(test_list) as f:
        test_list = f.read().splitlines()
    if prefix is not None:
        train_list = map(lambda x: os.path.join(prefix, x), train_list)
        test_list = map(lambda x: os.path.join(prefix, x), test_list)
    return train_list, test_list

def load_img(filename, flip = False, grayscale = False):
    if grayscale:
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(filename)
    if flip:
        img = cv2.flip(img, 1)
    if grayscale:
        width, height = img.shape
    else:
        width, height, _ = img.shape
    factor = max(256.0 / width, 256.0 / height)
    img = cv2.resize(img, None, fx = factor, fy = factor)
    return img

def load_imgs(filenames, reverse = False, flip = False, grayscale = False):
    if reverse:
        filenames = filenames[::-1]
    imgs = map(lambda x: load_img(x, flip, grayscale), filenames)
    return imgs

def load_video(video, data_dir, mask = None, reverse = False, flip = False, grayscale = False):
    filenames = os.listdir(os.path.join(data_dir, video))
    filenames = filter(lambda x: x[-4:] == 'jpeg', filenames)
    if mask is not None:
        filenames = filter(mask, filenames)
    filenames.sort()
    filenames = map(lambda x: os.path.join(data_dir, video, x), filenames)
    imgs = load_imgs(filenames, reverse, flip, grayscale)
    return imgs

def is_forward(video):
    if video[0] == 'F':
        return True
    return False

def select(imgs, n = 5):
    mags = [None] * (len(imgs)/2)
    for i in range(len(imgs)/2):
        mag, _ = cv2.cartToPolar(np.asarray(imgs[2*i], dtype = np.float32), np.asarray(imgs[2*i+1], dtype = np.float32))
    mags[i] = cv2.norm(mag)
    mags = np.asarray(mags)
    idx = np.argsort(mags)[::-1][:n]
    idx.sort()
    return idx

In [2]:
def load_flows(video, direction):
    imgs_ = load_video(video, './data/ArrowDataAll', 
                               mask = lambda x: x[:3] == 'of' + direction, 
                               grayscale=True)
    imgs_ = map(lambda x: cv2.resize(x, (227, 227)), imgs_)
    return imgs_

In [3]:
# display plots in this notebook
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [4]:
def load_features(video):
    with open(os.path.join('./data/ArrowDataAll', video, 'features3.csv'), 'r') as f:
        X = np.loadtxt(f, delimiter = ',')
    with open(os.path.join('./data/ArrowDataAll', video, 'labels3.csv'), 'r') as f:
        y = np.loadtxt(f, delimiter = ',')
    video_names_with_multitude = [video] * len(y)
    return X, y, video_names_with_multitude

In [5]:
train_list, test_list = load_list('./data', dataset = 1)

In [6]:
X_train = np.empty((0, 4096*3))
y_train = np.empty(0)
video_train = []
for video in train_list:
    print train_list.index(video)+1
    X, y, video_names_with_multitude = load_features(video)
    X_train = np.append(X_train, X, axis = 0)
    y_train = np.append(y_train, y, axis = 0)
    video_train.extend(video_names_with_multitude)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [7]:
X_test = np.empty((0, 4096*3))
y_test = np.empty(0)
video_test = []
for video in test_list:
    print test_list.index(video)+1
    X, y, video_names_with_multitude = load_features(video)
    X_test = np.append(X_test, X, axis = 0)
    y_test = np.append(y_test, y, axis = 0)
    video_test.extend(video_names_with_multitude)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [8]:
from sklearn.svm import SVC
svc = SVC(kernel = 'rbf', C = 0.1)
svc.fit(X_train, y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [18]:
print svc.score(X_train, y_train)

0.999305555556


In [9]:
print svc.score(X_test, y_test)

0.515277777778


In [20]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs = -1, n_estimators=10)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [21]:
print rfc.score(X_test, y_test)

0.488888888889


In [10]:
y_predict = svc.predict(X_test)

In [11]:
import pandas as pd
from pandas import Series, DataFrame

In [12]:
df = DataFrame(Series(y_test))

In [13]:
df[1] = Series(y_predict)
df[2] = Series(video_test)

In [14]:
df.columns = ['true', 'predict', 'video']

In [15]:
df

,true,predict,video
0,1.0,1.0,F_ebcfFg2ihYw
1,1.0,1.0,F_ebcfFg2ihYw
2,1.0,1.0,F_ebcfFg2ihYw
3,1.0,1.0,F_ebcfFg2ihYw
4,1.0,1.0,F_ebcfFg2ihYw
5,1.0,1.0,F_ebcfFg2ihYw
6,0.0,1.0,F_ebcfFg2ihYw
7,0.0,1.0,F_ebcfFg2ihYw
8,0.0,1.0,F_ebcfFg2ihYw
9,0.0,1.0,F_ebcfFg2ihYw


In [16]:
fold = [['A']*3 + ['B']*3 + ['C']*3 + ['D']*3]*60

In [17]:
import itertools
fold = list(itertools.chain(*fold))

In [18]:
df['fold'] = Series(fold)

In [19]:
df

,true,predict,video,fold
0,1.0,1.0,F_ebcfFg2ihYw,A
1,1.0,1.0,F_ebcfFg2ihYw,A
2,1.0,1.0,F_ebcfFg2ihYw,A
3,1.0,1.0,F_ebcfFg2ihYw,B
4,1.0,1.0,F_ebcfFg2ihYw,B
5,1.0,1.0,F_ebcfFg2ihYw,B
6,0.0,1.0,F_ebcfFg2ihYw,C
7,0.0,1.0,F_ebcfFg2ihYw,C
8,0.0,1.0,F_ebcfFg2ihYw,C
9,0.0,1.0,F_ebcfFg2ihYw,D


In [34]:
dfg = df.groupby(['video', 'fold']).mean()

In [35]:
dfg = dfg.unstack()

In [36]:
dfg

true                  predict                         
fold             A    B    C    D         A         B    C         D
video                                                               
B1ADMcXAQCzo0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B1RsaXI-SKchA  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B1U5XS34QmUuI  0.0  0.0  1.0  1.0  0.666667  0.666667  1.0  1.000000
B1mOABN2abDq0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B1uglCr8ueyv8  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B2ADMcXAQCzo0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B301TL9bUWr6I  0.0  0.0  1.0  1.0  1.000000  0.333333  1.0  1.000000
B401TL9bUWr6I  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
B5X6jprOZ29wY  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000
F_05gGCvIopwE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_2NOQEJKLqks  1.0  1.0  0.0  0.0  1.000000  1.000000  0.0  0.000000
F_3DL_97x0dpQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_3c66YWCXdbU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_69RNNex-sig  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_6ClKTIhJizQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_6d6oBTmEbRM  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_94HqFht60Ek  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_EIh4VpXQ8HU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_EbU7VPY4QIA  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_FaAjPRKXXv0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_FriaMvobUi0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_G4VKPR67xYs  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_HFkYJofKBis  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_IBp5Aip2Vw0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_MTXhdrkEtoE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_O2VWbZIrMuw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_PosKL3to6l8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_Rv0sbR0lbDg  1.0  1.0  0.0  0.0  1.000000  0.666667  1.0  0.333333
F_SyO3YwrsJPg  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_W8X7EFqVqNc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_XDpOtZ-sdQE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F__18hsqaDrHs  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F__RiKnRbBhtQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_aDEecz10DnY  1.0  1.0  0.0  0.0  0.666667  1.000000  1.0  1.000000
F_b-MbEx1Iqj8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_bYepd01KDFw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_cCMzlrpWHgo  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_dhfcutcvsLU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_ebcfFg2ihYw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_fPZNvBcoOtE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_fc8LngsvxUc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_gQXKFNJBgIc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_hyGJBV1xnJI  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_jmgV3OFn0aE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_k7viIK45xGQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_kJGQy-j3Nx4  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_kWetGqX9uFc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_kxCEpfTSfJU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_lanCzRFNoy8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_n1cCs-S5EKc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_n4MsOeSmwbs  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_oB2W0HIgpHg  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_pZcB4oJ5jsc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_pdy9isqFDxE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_qVH7ZUbowMU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000
F_tfMMAwlD7g

In [37]:
dfg['gt'] = dfg.apply(lambda x: x['true']['A'] + x['true']['B']- x['true']['C']-x['true']['D'], axis = 1)

In [38]:
dfg['my'] = dfg.apply(lambda x: x['predict']['A'] + x['predict']['B']- x['predict']['C']-x['predict']['D'], axis = 1)

In [39]:
dfg

true                  predict                            gt  \
fold             A    B    C    D         A         B    C         D        
video                                                                       
B1ADMcXAQCzo0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B1RsaXI-SKchA  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B1U5XS34QmUuI  0.0  0.0  1.0  1.0  0.666667  0.666667  1.0  1.000000 -2.0   
B1mOABN2abDq0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B1uglCr8ueyv8  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B2ADMcXAQCzo0  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B301TL9bUWr6I  0.0  0.0  1.0  1.0  1.000000  0.333333  1.0  1.000000 -2.0   
B401TL9bUWr6I  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
B5X6jprOZ29wY  0.0  0.0  1.0  1.0  1.000000  1.000000  1.0  1.000000 -2.0   
F_05gGCvIopwE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_2NOQEJKLqks  1.0  1.0  0.0  0.0  1.000000  1.000000  0.0  0.000000  2.0   
F_3DL_97x0dpQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_3c66YWCXdbU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_69RNNex-sig  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_6ClKTIhJizQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_6d6oBTmEbRM  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_94HqFht60Ek  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_EIh4VpXQ8HU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_EbU7VPY4QIA  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_FaAjPRKXXv0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_FriaMvobUi0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_G4VKPR67xYs  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_HFkYJofKBis  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_IBp5Aip2Vw0  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_MTXhdrkEtoE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_O2VWbZIrMuw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_PosKL3to6l8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_Rv0sbR0lbDg  1.0  1.0  0.0  0.0  1.000000  0.666667  1.0  0.333333  2.0   
F_SyO3YwrsJPg  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_W8X7EFqVqNc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_XDpOtZ-sdQE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F__18hsqaDrHs  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F__RiKnRbBhtQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_aDEecz10DnY  1.0  1.0  0.0  0.0  0.666667  1.000000  1.0  1.000000  2.0   
F_b-MbEx1Iqj8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_bYepd01KDFw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_cCMzlrpWHgo  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_dhfcutcvsLU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_ebcfFg2ihYw  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_fPZNvBcoOtE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_fc8LngsvxUc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_gQXKFNJBgIc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_hyGJBV1xnJI  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_jmgV3OFn0aE  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_k7viIK45xGQ  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_kJGQy-j3Nx4  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_kWetGqX9uFc  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_kxCEpfTSfJU  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_lanCzRFNoy8  1.0  1.0  0.0  0.0  1.000000  1.000000  1.0  1.000000  2.0   
F_n1cCs-S5

In [44]:
video = 'B101TL9bUWr6I'
y = np.empty(0)
for reverse in [False, True]:
        imgs_ = load_video(video, './data/ArrowDataAll/', flip = flip, reverse = reverse, mask = lambda x: x[:2] == 'of', grayscale = True)
        imgs_ = map(lambda x: cv2.resize(x, (227, 227)), imgs_)
        imgs = []
        for i in range(len(imgs_) / 2):
            imgs.append(np.stack([imgs_[2*i], imgs_[2*i+1]], axis = 0))
        imgs = np.asarray(imgs)
        break


In [45]:
imgs.shape

(476, 2, 227, 227)

In [46]:
imgs[0,1]

array([[122, 122, 122, ..., 122, 122, 122],
       [122, 122, 122, ..., 122, 122, 122],
       [122, 122, 122, ..., 122, 122, 122],
       ..., 
       [130, 130, 130, ..., 122, 122, 122],
       [130, 130, 130, ..., 122, 122, 122],
       [130, 130, 130, ..., 122, 122, 122]], dtype=uint8)

In [40]:
imgs.shape

NameError: name 'imgs' is not defined